# https://www.kaggle.com/thehemen/pytorch-transfer-learning-with-k-folds-by-drug-ids by Vladimir Zhuravlev 
# https://www.kaggle.com/thehemen/pytorch-transfer-learning-with-k-folds-by-drug-ids by Vladimir Zhuravlev 
# https://www.kaggle.com/thehemen/pytorch-transfer-learning-with-k-folds-by-drug-ids by Vladimir Zhuravlev 
# https://www.kaggle.com/thehemen/pytorch-transfer-learning-with-k-folds-by-drug-ids by Vladimir Zhuravlev 
# https://www.kaggle.com/thehemen/pytorch-transfer-learning-with-k-folds-by-drug-ids by Vladimir Zhuravlev 
# https://www.kaggle.com/thehemen/pytorch-transfer-learning-with-k-folds-by-drug-ids by Vladimir Zhuravlev 
# https://www.kaggle.com/thehemen/pytorch-transfer-learning-with-k-folds-by-drug-ids by Vladimir Zhuravlev 
# https://www.kaggle.com/thehemen/pytorch-transfer-learning-with-k-folds-by-drug-ids by Vladimir Zhuravlev 
# https://www.kaggle.com/thehemen/pytorch-transfer-learning-with-k-folds-by-drug-ids by Vladimir Zhuravlev 
# https://www.kaggle.com/thehemen/pytorch-transfer-learning-with-k-folds-by-drug-ids by Vladimir Zhuravlev 
# https://www.kaggle.com/thehemen/pytorch-transfer-learning-with-k-folds-by-drug-ids by Vladimir Zhuravlev 
# https://www.kaggle.com/thehemen/pytorch-transfer-learning-with-k-folds-by-drug-ids by Vladimir Zhuravlev 

In [ ]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns

from datetime import timedelta
from time import time
from types import SimpleNamespace

import joblib
 
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.modules.loss import _WeightedLoss
 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install /kaggle/input/iterative-stratification/iterative-stratification-master/
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
data_dir = '../input/lish-moa/'
os.listdir(data_dir)

In [ ]:
def set_columns(state, train_features):
    state.GENES = [col for col in train_features.columns if col.startswith('g-')]
    state.CELLS = [col for col in train_features.columns if col.startswith('c-')]

    print('GENES: {}'.format(state.GENES[:10]))
    print('CELLS: {}'.format(state.CELLS[:10]))

In [ ]:
def mix_seed(z):
    # splitmix32
    z = (z + 0x9e3779b9) & 0xffffffff
    z = ((z ^ (z >> 15))*0x85ebca6b) & 0xffffffff
    z = ((z ^ (z >> 13))*0xc2b2ae35) & 0xffffffff
    return z ^ (z >> 16)


def set_parameters(state):
    # HyperParameters
    state.SEEDS = list(map(mix_seed, range(10)))
    state.NFOLDS = 7
    state.DRUG_THRESH = 18

    state.DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
    state.EPOCHS = 24
    state.BATCH_SIZE = 128

    state.WEIGHT_DECAY = {'ALL_TARGETS': 1e-5, 'SCORED_ONLY': 3e-6}
    state.MAX_LR = {'ALL_TARGETS': 1e-2, 'SCORED_ONLY': 3e-3}
    state.DIV_FACTOR = {'ALL_TARGETS': 1e3, 'SCORED_ONLY': 1e2}
    state.PCT_START = 0.1

# RankGauss

In [ ]:
def set_rank_gauss(state, train_features):
    state.col_to_transformer = {}
    for col in (state.GENES + state.CELLS):
        transformer = QuantileTransformer(n_quantiles=100, random_state=0, output_distribution="normal")
        raw_vec = train_features[col].values.reshape(-1, 1)
        transformer.fit(raw_vec)
        state.col_to_transformer[col] = transformer # TODO deepcopy


def apply_rank_gauss(state, data_set):
    for col, transformer in state.col_to_transformer.items():
        data_set[col] = transformer.transform(data_set[col].values.reshape(-1, 1)).ravel()

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


def set_seed(state):
    state.SEED_VALUE = 42
    seed_everything(seed=state.SEED_VALUE)

# PCA features + Existing features

In [ ]:
def set_pca_genes(state, train_features, test_features):
    n_comp = 600
    GENES = state.GENES
    SEED_VALUE = state.SEED_VALUE
    
    data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
    transformer = PCA(n_components=n_comp, random_state=SEED_VALUE)
    transformer.fit(data[GENES])
    
    state.pca_genes_n_comp = n_comp
    state.pca_genes = transformer


def apply_pca_genes(state, data_set):
    n_comp = state.pca_genes_n_comp
    GENES = state.GENES
    transformer = state.pca_genes
    
    data2 = transformer.transform(data_set[GENES])
    data2 = pd.DataFrame(data2, columns=[f'pca_G-{i}' for i in range(n_comp)])

    data2_features = pd.concat((data_set, data2), axis=1)
    print('GENES data2_features: {}'.format(data2_features.shape))
    return data2_features

In [ ]:
def set_pca_cells(state, train_features, test_features):
    n_comp = 50
    CELLS = state.CELLS
    SEED_VALUE = state.SEED_VALUE
    
    data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
    transformer = PCA(n_components=n_comp, random_state=SEED_VALUE)
    transformer.fit(data[CELLS])
    
    state.pca_cells_n_comp = n_comp
    state.pca_cells = transformer


def apply_pca_cells(state, data_set):
    n_comp = state.pca_cells_n_comp
    CELLS = state.CELLS
    transformer = state.pca_cells
    
    data2 = transformer.transform(data_set[CELLS])
    data2 = pd.DataFrame(data2, columns=[f'pca_C-{i}' for i in range(n_comp)])

    data2_features = pd.concat((data_set, data2), axis=1)
    print('CELLS data2_features: {}'.format(data2_features.shape))
    return data2_features

# feature Selection using Variance Encoding

In [ ]:
def set_var_thresh(state, train_features, test_features):
    var_thresh = VarianceThreshold(0.8)
    data = train_features.append(test_features)
    var_thresh.fit(data.iloc[:, 4:])
    state.var_thresh = var_thresh


def apply_var_thresh(state, data_set):
    xcols = ['sig_id','cp_type','cp_time','cp_dose']
    var_thresh = state.var_thresh
    data_features_transformed = var_thresh.transform(data_set.iloc[:, 4:])
    data_features = pd.DataFrame(data_set[xcols].values.reshape(-1, 4), columns=xcols)
    data_features = pd.concat([data_features, pd.DataFrame(data_features_transformed)], axis=1)
    print('VAR THRESH data_features: {}'.format(data_features.shape))
    return data_features

In [ ]:
def apply_train_extensions(train_set, train_targets_scored, train_targets_nonscored, train_drug):
    train = train_set.merge(train_targets_scored, on='sig_id')
    train = train.merge(train_targets_nonscored, on='sig_id')
    train = train.merge(train_drug, on='sig_id')
    print('EXTENSION train: {}'.format(train.shape))
    return train


def apply_cleaning(data_set):
    data_set = data_set[data_set['cp_type'] != 'ctl_vehicle'].reset_index(drop=True)
    data_set = data_set.drop('cp_type', axis=1)
    print('CLEANING data_set: {}'.format(data_set.shape))
    return data_set


def apply_dummies(data_set):
    # former process_data
    data_set = pd.get_dummies(data_set, columns=['cp_time', 'cp_dose'])
    print('DUMMIES data_set: {}'.format(data_set.shape))    
    return data_set

In [ ]:
def set_meta(state, train, train_targets_scored, train_targets_nonscored):
    target_cols = [x for x in train_targets_scored.columns if x != 'sig_id']
    aux_target_cols = [x for x in train_targets_nonscored.columns if x != 'sig_id']
    all_target_cols = target_cols + aux_target_cols

    num_targets = len(target_cols)
    num_aux_targets = len(aux_target_cols)
    num_all_targets = len(all_target_cols)

    print('META num_targets: {}'.format(num_targets))
    print('META num_aux_targets: {}'.format(num_aux_targets))
    print('META num_all_targets: {}'.format(num_all_targets))
    
    state.target_cols = target_cols
    state.aux_target_cols = aux_target_cols
    state.all_target_cols = all_target_cols
    state.num_targets = num_targets
    state.num_aux_targets = num_aux_targets
    state.num_all_targets = num_all_targets
    
    feature_cols = [c for c in train.columns if c not in all_target_cols]
    feature_cols = [c for c in feature_cols if c not in ['kfold', 'sig_id', 'drug_id']]
    
    state.feature_cols = feature_cols
    state.num_features = len(feature_cols)
    print("META num_features", state.num_features)

In [ ]:
def make_cv_folds(state, train_set):
    DRUG_THRESH = state.DRUG_THRESH
    SEEDS = state.SEEDS
    NFOLDS = state.NFOLDS
    target_cols = state.target_cols
    
    vc = train_set.drug_id.value_counts()
    vc1 = vc.loc[vc <= DRUG_THRESH].index.sort_values()
    vc2 = vc.loc[vc > DRUG_THRESH].index.sort_values()

    for seed_id in SEEDS:
        kfold_col = 'kfold_{}'.format(seed_id)

        # STRATIFY DRUGS 18X OR LESS
        dct1 = {}
        dct2 = {}

        skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed_id)
        tmp = train_set.groupby('drug_id')[target_cols].mean().loc[vc1]

        for fold,(idxT, idxV) in enumerate(skf.split(tmp, tmp[target_cols])):
            dd = {k: fold for k in tmp.index[idxV].values}
            dct1.update(dd)

        # STRATIFY DRUGS MORE THAN 18X
        skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed_id)
        tmp = train_set.loc[train_set.drug_id.isin(vc2)].reset_index(drop=True)

        for fold,(idxT, idxV) in enumerate(skf.split(tmp, tmp[target_cols])):
            dd = {k: fold for k in tmp.sig_id[idxV].values}
            dct2.update(dd)

        # ASSIGN FOLDS
        train_set[kfold_col] = train_set.drug_id.map(dct1)
        train_set.loc[train_set[kfold_col].isna(), kfold_col] = train_set.loc[train_set[kfold_col].isna(), 'sig_id'].map(dct2)
        train_set[kfold_col] = train_set[kfold_col].astype('int8')

    return train_set

In [ ]:
def set_all(state, train_features, test_features,
            train_targets_scored, train_targets_nonscored, train_drug):
    """ Create input transformations. """
    train_features = train_features.copy()
    print(f"ALL train_features {train_features.shape!r}")
    test_features = test_features.copy()
    print(f"ALL test_features {test_features.shape!r}")
    
    # parameteres
    set_parameters(state)
    
    # column meta
    set_columns(state, train_features)
    
    # quantile transform
    set_rank_gauss(state, train_features)
    apply_rank_gauss(state, train_features)
    apply_rank_gauss(state, test_features)
    
    # seed here for some reason
    set_seed(state)
    
    # pca features
    set_pca_genes(state, train_features, test_features)
    train_features = apply_pca_genes(state, train_features)
    test_features = apply_pca_genes(state, test_features)
    
    set_pca_cells(state, train_features, test_features)
    train_features = apply_pca_cells(state, train_features)
    test_features = apply_pca_cells(state, test_features)
    
    # variance threshold
    set_var_thresh(state, train_features, test_features)
    train_features = apply_var_thresh(state, train_features)
    test_features = apply_var_thresh(state, test_features)
    
    # add targets & drug
    train = apply_train_extensions(train_features, train_targets_scored, train_targets_nonscored, train_drug)
    
    # strip out waste
    train = apply_cleaning(train)
    test = apply_cleaning(test_features)
    
    # dummies
    train = apply_dummies(train)
    test = apply_dummies(test)
    
    # put other variables into state
    set_meta(state, train, train_targets_scored, train_targets_nonscored)
    
    # persist
    joblib.dump(state, "state_object.joblib")

In [ ]:
def apply_all(state, data_set):
    data_set = data_set.copy()
    apply_rank_gauss(state, data_set)
    data_set = apply_pca_genes(state, data_set)
    data_set = apply_pca_cells(state, data_set)
    data_set = apply_var_thresh(state, data_set)
    data_set = apply_cleaning(data_set)
    data_set = apply_dummies(data_set)
    print(f"APPLY data_set {data_set.shape!r}")
    return data_set


def apply_all_train(state, data_set,
                    train_targets_scored, train_targets_nonscored, train_drug):
    data_set = data_set.copy()
    apply_rank_gauss(state, data_set)
    data_set = apply_pca_genes(state, data_set)
    data_set = apply_pca_cells(state, data_set)
    data_set = apply_var_thresh(state, data_set)
    data_set = apply_train_extensions(data_set, train_targets_scored, train_targets_nonscored, train_drug)
    data_set = apply_cleaning(data_set)
    data_set = make_cv_folds(state, data_set)
    data_set = apply_dummies(data_set)
    print(f"APPLY TRAIN data_set {data_set.shape!r}")
    return data_set

# Dataset Classes

In [ ]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)
        }
        
        return dct


class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }

        return dct

In [ ]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()

        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)

        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    return final_loss, valid_preds


def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    return preds

In [ ]:
class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1

        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
            
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

# Model

In [ ]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets):
        super(Model, self).__init__()
        self.hidden_size = [1504, 1248, 992, 736] # XXXGBoost change (round to multiple of 32)
        self.dropout_value = [0.5, 0.35, 0.3, 0.25]

        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.Linear(num_features, self.hidden_size[0])
        
        self.batch_norm2 = nn.BatchNorm1d(self.hidden_size[0])
        self.dropout2 = nn.Dropout(self.dropout_value[0])
        self.dense2 = nn.Linear(self.hidden_size[0], self.hidden_size[1])

        self.batch_norm3 = nn.BatchNorm1d(self.hidden_size[1])
        self.dropout3 = nn.Dropout(self.dropout_value[1])
        self.dense3 = nn.Linear(self.hidden_size[1], self.hidden_size[2])

        self.batch_norm4 = nn.BatchNorm1d(self.hidden_size[2])
        self.dropout4 = nn.Dropout(self.dropout_value[2])
        self.dense4 = nn.Linear(self.hidden_size[2], self.hidden_size[3])

        self.batch_norm5 = nn.BatchNorm1d(self.hidden_size[3])
        self.dropout5 = nn.Dropout(self.dropout_value[3])
        self.dense5 = nn.utils.weight_norm(nn.Linear(self.hidden_size[3], num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))

        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = F.leaky_relu(self.dense3(x))

        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = F.leaky_relu(self.dense4(x))

        x = self.batch_norm5(x)
        x = self.dropout5(x)
        x = self.dense5(x)
        return x


class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)

        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
            
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))    

In [ ]:
class FineTuneScheduler:
    def __init__(self, epochs, device):
        self.epochs = epochs
        self.device = device
        self.epochs_per_step = 0
        self.frozen_layers = []

    def copy_without_top(self, model, num_features, num_targets, num_targets_new):
        self.frozen_layers = []

        model_new = Model(num_features, num_targets)
        model_new.load_state_dict(model.state_dict())

        # Freeze all weights
        for name, param in model_new.named_parameters():
            layer_index = name.split('.')[0][-1]

            if layer_index == 5:
                continue

            param.requires_grad = False

            # Save frozen layer names
            if layer_index not in self.frozen_layers:
                self.frozen_layers.append(layer_index)

        self.epochs_per_step = max(1, self.epochs // len(self.frozen_layers))

        # Replace the top layers with another ones
        model_new.batch_norm5 = nn.BatchNorm1d(model_new.hidden_size[3])
        model_new.dropout5 = nn.Dropout(model_new.dropout_value[3])
        model_new.dense5 = nn.utils.weight_norm(nn.Linear(model_new.hidden_size[-1], num_targets_new))
        model_new.to(self.device)
        return model_new

    def step(self, epoch, model):
        if len(self.frozen_layers) == 0:
            return

        if epoch % self.epochs_per_step == 0:
            last_frozen_index = self.frozen_layers[-1]
            
            # Unfreeze parameters of the last frozen layer
            for name, param in model.named_parameters():
                layer_index = name.split('.')[0][-1]

                if layer_index == last_frozen_index:
                    param.requires_grad = True

            del self.frozen_layers[-1]  # Remove the last layer as unfrozen

# Single fold training

In [ ]:
def run_training(state, fold_id, seed_id, train_, test_):
    feature_cols = state.feature_cols
    target_cols = state.target_cols
    all_target_cols = state.all_target_cols
    num_features = state.num_features
    num_all_targets = state.num_all_targets
    num_targets = state.num_targets
    BATCH_SIZE = state.BATCH_SIZE
    PCT_START = state.PCT_START
    DIV_FACTOR = state.DIV_FACTOR
    WEIGHT_DECAY = state.WEIGHT_DECAY
    MAX_LR = state.MAX_LR
    EPOCHS = state.EPOCHS
    DEVICE = state.DEVICE
    
    seed_everything(seed_id)
    
    kfold_col = f'kfold_{seed_id}'
    trn_idx = train_[train_[kfold_col] != fold_id].index
    val_idx = train_[train_[kfold_col] == fold_id].index
    
    train_df = train_[train_[kfold_col] != fold_id].reset_index(drop=True)
    valid_df = train_[train_[kfold_col] == fold_id].reset_index(drop=True)
    
    def train_model(model, tag_name, target_cols_now, fine_tune_scheduler=None):
        x_train, y_train  = train_df[feature_cols].values, train_df[target_cols_now].values
        x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols_now].values
        
        train_dataset = MoADataset(x_train, y_train)
        valid_dataset = MoADataset(x_valid, y_valid)

        trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=WEIGHT_DECAY[tag_name])
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer,
                                                  steps_per_epoch=len(trainloader),
                                                  pct_start=PCT_START,
                                                  div_factor=DIV_FACTOR[tag_name], 
                                                  max_lr=MAX_LR[tag_name],
                                                  epochs=EPOCHS)
        
        loss_fn = nn.BCEWithLogitsLoss()
        loss_tr = SmoothBCEwLogits(smoothing=0.001)

        oof = np.zeros((len(train_), len(target_cols_now)))
        best_loss = np.inf
        
        for epoch in range(EPOCHS):
            if fine_tune_scheduler is not None:
                fine_tune_scheduler.step(epoch, model)

            train_loss = train_fn(model, optimizer, scheduler, loss_tr, trainloader, DEVICE)
            valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
            print(f"SEED: {seed_id}, FOLD: {fold_id}, {tag_name}, EPOCH: {epoch}, train_loss: {train_loss:.6f}, valid_loss: {valid_loss:.6f}")

            if np.isnan(valid_loss):
                break
            
            if valid_loss < best_loss:
                best_loss = valid_loss
                oof[val_idx] = valid_preds
                torch.save(model.state_dict(), f"{tag_name}_SEED{seed_id}_FOLD{fold_id}_.pth")

        return oof

    fine_tune_scheduler = FineTuneScheduler(EPOCHS, DEVICE)

    pretrained_model = Model(num_features, num_all_targets)
    pretrained_model.to(DEVICE)

    # Train on scored + nonscored targets
    train_model(pretrained_model, 'ALL_TARGETS', all_target_cols)

    # Load the pretrained model with the best loss
    pretrained_model = Model(num_features, num_all_targets)
    pretrained_model.load_state_dict(torch.load(f"ALL_TARGETS_SEED{seed_id}_FOLD{fold_id}_.pth"))
    pretrained_model.to(DEVICE)

    # Copy model without the top layer
    final_model = fine_tune_scheduler.copy_without_top(pretrained_model, num_features, num_all_targets, num_targets)

    # Fine-tune the model on scored targets only
    oof = train_model(final_model, 'SCORED_ONLY', target_cols, fine_tune_scheduler)

    # Load the fine-tuned model with the best loss
    model = Model(num_features, num_targets)
    model.load_state_dict(torch.load(f"SCORED_ONLY_SEED{seed_id}_FOLD{fold_id}_.pth"))
    model.to(DEVICE)

    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    predictions = inference_fn(model, testloader, DEVICE)
    return oof, predictions

In [ ]:
def run_k_fold(state, seed_id, train, test):
    target_cols = state.target_cols
    NFOLDS = state.NFOLDS
    
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold_id in range(NFOLDS):
        oof_, pred_ = run_training(state, fold_id, seed_id, train, test)
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [ ]:
def run_over_seeds(state, train_features, test_features, train_targets_scored, train_targets_nonscored, train_drug):
    target_cols = state.target_cols
    SEEDS = state.SEEDS
    NFOLDS = state.NFOLDS
    
    train = apply_all_train(state, train_features, train_targets_scored, train_targets_nonscored, train_drug)
    test = apply_all(state, test_features)
    
    # Averaging on multiple SEEDS
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))

    time_begin = time()

    for seed_id in SEEDS:
        oof_, predictions_ = run_k_fold(state, seed_id, train, test)
        oof += oof_ / len(SEEDS)
        predictions += predictions_ / len(SEEDS)

    time_diff = time() - time_begin
    print(timedelta(seconds=time_diff))

    train[target_cols] = oof
    test[target_cols] = predictions
    
    # cross-validation score
    valid_results = train_targets_scored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)

    y_true = train_targets_scored[target_cols].values
    y_pred = valid_results[target_cols].values

    score = 0.0

    for i in range(len(target_cols)):
        score += log_loss(y_true[:, i], y_pred[:, i])

    print("CV log_loss: ", score / y_pred.shape[1])
    joblib.dump((train, test), "train_test.joblib")

In [ ]:
train_features = pd.read_csv(data_dir + 'train_features.csv')
train_targets_scored = pd.read_csv(data_dir + 'train_targets_scored.csv')
train_targets_nonscored = pd.read_csv(data_dir + 'train_targets_nonscored.csv')
train_drug = pd.read_csv(data_dir + 'train_drug.csv')
test_features = pd.read_csv(data_dir + 'test_features.csv')
sample_submission = pd.read_csv(data_dir + 'sample_submission.csv')

print('train_features: {}'.format(train_features.shape))
print('train_targets_scored: {}'.format(train_targets_scored.shape))
print('train_targets_nonscored: {}'.format(train_targets_nonscored.shape))
print('train_drug: {}'.format(train_drug.shape))
print('test_features: {}'.format(test_features.shape))
print('sample_submission: {}'.format(sample_submission.shape))

In [ ]:
xxxtorch = SimpleNamespace()
set_all(xxxtorch, train_features, test_features, train_targets_scored, train_targets_nonscored, train_drug)

In [ ]:
run_over_seeds(xxxtorch, train_features, test_features, train_targets_scored, train_targets_nonscored, train_drug)